In [45]:
import os 
import torch
import numpy as np 
import pandas as pd 

from torch import nn, optim
from torch.utils.data import DataLoader # type: ignore
from torchvision import datasets, transforms

In [69]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

batchsize = 100
transform = transforms.ToTensor()

# Download the dataset directly
training_data = datasets.FashionMNIST(root="C:/Users/tejas/Downloads/archive", train=True, download=True, transform=transform)
test_data = datasets.FashionMNIST(root="C:/Users/tejas/Downloads/archive", train=False, download=True, transform=transform)

train_dataloader = DataLoader(training_data, batch_size=batchsize)
test_dataloader = DataLoader(test_data, batch_size=batchsize)

100%|██████████| 26.4M/26.4M [00:06<00:00, 4.03MB/s]


Extracting C:/Users/tejas/Downloads/archive\FashionMNIST\raw\train-images-idx3-ubyte.gz to C:/Users/tejas/Downloads/archive\FashionMNIST\raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 195kB/s]


Extracting C:/Users/tejas/Downloads/archive\FashionMNIST\raw\train-labels-idx1-ubyte.gz to C:/Users/tejas/Downloads/archive\FashionMNIST\raw



100%|██████████| 4.42M/4.42M [00:01<00:00, 3.08MB/s]


Extracting C:/Users/tejas/Downloads/archive\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to C:/Users/tejas/Downloads/archive\FashionMNIST\raw



100%|██████████| 5.15k/5.15k [00:00<?, ?B/s]

Extracting C:/Users/tejas/Downloads/archive\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to C:/Users/tejas/Downloads/archive\FashionMNIST\raw



In [27]:
# define hyperparmaters
sequence_len = 28
input_len = 28
hidden_size = 128
num_layers = 2
num_classes = 10
num_epochs = 5
learning_rate = 0.01

In [28]:
class LSTM(nn.Module):
    def __init__(self, input_len, hidden_size, num_class, num_layers):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_len, hidden_size, num_layers, batch_first=True)
        self.output_layer = nn.Linear(hidden_size, num_classes)
    
    def forward(self, X):
        hidden_states = torch.zeros(self.num_layers, X.size(0), self.hidden_size)
        cell_states = torch.zeros(self.num_layers, X.size(0), self.hidden_size)
        out, _ = self.lstm(X, (hidden_states, cell_states))
        out = self.output_layer(out[:, -1, :])
        return out 

In [41]:
model = LSTM(input_len, hidden_size, num_classes, num_layers)
print(model)

LSTM(
  (lstm): LSTM(28, 128, num_layers=2, batch_first=True)
  (output_layer): Linear(in_features=128, out_features=10, bias=True)
)


In [46]:
loss_func = nn.CrossEntropyLoss()
sgd = optim.SGD(model.parameters(), lr=learning_rate)
adam = optim.Adam(model.parameters(), lr=learning_rate)

In [47]:
def train(num_epochs, model, train_dataloader, loss_func, optimizer):
    total_steps = len(train_dataloader)

    for epoch in range(num_epochs):
        for batch, (images, labels) in enumerate(train_dataloader):
            images = images.reshape(-1, sequence_len, input_len)
            
            output = model(images)
            loss = loss_func(output, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (batch+1)%100 == 0:
                print(f"Epoch: {epoch+1}; Batch {batch+1} / {total_steps}; Loss: {loss.item():>4f}")

In [70]:
train(num_epochs, model, train_dataloader, loss_func, adam)

Epoch: 1; Batch 100 / 600; Loss: 0.858839
Epoch: 1; Batch 200 / 600; Loss: 0.951888
Epoch: 1; Batch 300 / 600; Loss: 0.458994
Epoch: 1; Batch 400 / 600; Loss: 0.620316
Epoch: 1; Batch 500 / 600; Loss: 0.621738
Epoch: 1; Batch 600 / 600; Loss: 0.394197
Epoch: 2; Batch 100 / 600; Loss: 0.413824
Epoch: 2; Batch 200 / 600; Loss: 0.438546
Epoch: 2; Batch 300 / 600; Loss: 0.343214
Epoch: 2; Batch 400 / 600; Loss: 0.393300
Epoch: 2; Batch 500 / 600; Loss: 0.527712
Epoch: 2; Batch 600 / 600; Loss: 0.264534
Epoch: 3; Batch 100 / 600; Loss: 0.339338
Epoch: 3; Batch 200 / 600; Loss: 0.335894
Epoch: 3; Batch 300 / 600; Loss: 0.306062
Epoch: 3; Batch 400 / 600; Loss: 0.304676
Epoch: 3; Batch 500 / 600; Loss: 0.486916
Epoch: 3; Batch 600 / 600; Loss: 0.249519
Epoch: 4; Batch 100 / 600; Loss: 0.320727
Epoch: 4; Batch 200 / 600; Loss: 0.318134
Epoch: 4; Batch 300 / 600; Loss: 0.328715
Epoch: 4; Batch 400 / 600; Loss: 0.319100
Epoch: 4; Batch 500 / 600; Loss: 0.470505
Epoch: 4; Batch 600 / 600; Loss: 0

In [71]:
test_images, test_labels = next(iter(test_dataloader))
test_labels

tensor([9, 2, 1, 1, 6, 1, 4, 6, 5, 7, 4, 5, 7, 3, 4, 1, 2, 4, 8, 0, 2, 5, 7, 9,
        1, 4, 6, 0, 9, 3, 8, 8, 3, 3, 8, 0, 7, 5, 7, 9, 6, 1, 3, 7, 6, 7, 2, 1,
        2, 2, 4, 4, 5, 8, 2, 2, 8, 4, 8, 0, 7, 7, 8, 5, 1, 1, 2, 3, 9, 8, 7, 0,
        2, 6, 2, 3, 1, 2, 8, 4, 1, 8, 5, 9, 5, 0, 3, 2, 0, 6, 5, 3, 6, 7, 1, 8,
        0, 1, 4, 2])

In [72]:
test_output = model(test_images.view(-1, 28, 28))

In [73]:
predicted = torch.max(test_output, 1)[1]
predicted

tensor([9, 2, 1, 1, 6, 1, 4, 6, 5, 7, 4, 5, 5, 3, 4, 1, 2, 4, 8, 0, 2, 7, 7, 5,
        1, 2, 6, 0, 9, 3, 8, 8, 3, 3, 8, 0, 7, 5, 7, 9, 0, 1, 0, 7, 6, 7, 2, 1,
        4, 2, 4, 2, 5, 8, 2, 2, 8, 4, 8, 0, 7, 7, 8, 5, 1, 1, 3, 3, 7, 8, 7, 0,
        8, 6, 4, 3, 1, 2, 8, 4, 1, 8, 5, 9, 5, 0, 3, 2, 0, 2, 5, 3, 6, 7, 1, 8,
        0, 1, 4, 2])

In [74]:
correct = [1 for i in range(100) if predicted[i] == test_labels[i]]

In [75]:
percentage_correct = sum(correct)/100

In [76]:
percentage_correct

0.87

In [77]:
def test_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            # reshape images
            X = X.reshape(-1, 28, 28)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f"Test Error:\n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")
    return 100*correct

In [78]:
test_loop(test_dataloader, model, loss_func, adam)

Test Error:
 Accuracy: 87.9%, Avg loss: 0.322271



87.88